In [1]:
#!pip install azureml-sdk

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of azureml-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of azureml-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ---------------------------------------- 0.0/306.7 kB ? eta -:--:--
     ------- ------------------------------- 61.4/306.7 kB 3.4 MB/s eta 0:00:01
     ------------------------------- ------ 256.0/306.7 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 306.7/306.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (se


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [2]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [3]:
print(resource_group)
print(workspace_name)
print(region)

deploy-ml-model-azure-rg
sklearnmodel-ws
eastus


In [4]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying StorageAccount with name sklearnmstorage628798364.
Deploying KeyVault with name sklearnmkeyvault472c5574.
Deploying AppInsights with name sklearnminsights94b4950f.
Deployed AppInsights with name sklearnminsights94b4950f. Took 113.71 seconds.
Deploying Workspace with name sklearnmodel-ws.
Deployed Workspace with name sklearnmodel-ws. Took 23.16 seconds.
Workspace sklearnmodel-ws created


In [5]:
# Specify the path to your  model file
model_path = 'logistic_regression_model.pkl'

In [6]:
model_name='titanic_prediction_model'

In [7]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model titanic_prediction_model


In [8]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [9]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [10]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [11]:
service = Model.deploy(workspace=ws,
                       name='titanic-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\paawa\AppData\Local\Temp\ipykernel_7220\2771400049.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-10-07 15:10:01+03:00 Creating Container Registry if not exists..
2024-10-07 15:20:02+03:00 Registering the environment.
2024-10-07 15:20:05+03:00 Building image..
2024-10-07 15:31:21+03:00 Generating deployment configuration.
2024-10-07 15:31:23+03:00 Submitting deployment to compute..
2024-10-07 15:31:32+03:00 Checking the status of deployment titanic-prediction-service..
2024-10-07 15:32:49+03:00 Checking the status of inference endpoint titanic-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [12]:
scoring_uri = service.scoring_uri

In [13]:
scoring_uri

'http://3258e8bc-4b20-4edd-bd8e-6c8502346626.eastus.azurecontainer.io/score'

In [14]:
# Delete the service

service_name = "titanic-prediction-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-10-07 15:52:31+03:00 Deleting service.
2024-10-07 15:52:33+03:00 Deleting service entity.
Succeeded


In [16]:
# Specify the name of the registered model to delete
model_name = 'titanic_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()

In [17]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)